## This script is to convert recorded demonstrations (from rosbag format) to samples (in txt format) to conform with the TP-GMM code.

In [13]:
import rosbag
import numpy as np
from math import sqrt
# from franka_msgs.msg import FrankaState

# main_dir = "/home/zizo/Disassembly Teleop/"
# TPGMM_dir = "/home/zizo/myGithubRepos/Task-Parameterized-Gaussian-Mixture-Model/"
main_dir = "C:/Users/AWasf/OneDrive/Desktop/PhD Lab Work/Disassembly-Teleop/"
TPGMM_dir = "C:/Users/AWasf/OneDrive/Desktop/PhD Lab Work/Task-Parameterized-Gaussian-Mixture-Model/"

### 2D

In [8]:
task_name = "Rcover"
task_num = str(3)

demon_bag = rosbag.Bag(main_dir + "Demons/" + task_name + "/action/Franka_Stage2_" + task_name + "_action_Demon" + task_num + ".bag", 'r')

data_topic = "/panda_teleop/follower_state_controller/franka_states"

conc_arr_Data = np.zeros((3,1))
msg_count = 0
for topic, msg, t in demon_bag.read_messages(data_topic):

    # Preparing the Data points matrix from the Demons
    # arr_Data = np.array([[msg.O_T_EE[12]], [msg.O_T_EE[13]], [msg.O_T_EE[14]]]) # TODO: add the erro_pose value
    arr_Data = np.array([[sqrt(msg.O_T_EE_c[12]**2 + msg.O_T_EE_c[13]**2 + msg.O_T_EE_c[14]**2)], [msg.O_T_EE[12]], [msg.O_T_EE[14]]]) # Euc. distance of error_pose

    conc_arr_Data = np.concatenate((conc_arr_Data, arr_Data), axis=1)

    # Preparing the b matrix & A matrix for frame 1 (Position & RotationMatrix of the Starting point(frame) of the Demon w.r.t the base frame of the robot (panda_link0))
    if (msg_count == 0):
        # arr_frame1_b = np.array([[msg.O_T_EE[12]], [msg.O_T_EE[13]], [msg.O_T_EE[14]]])
        # arr_frame1_A = np.matrix([[msg.O_T_EE[0], msg.O_T_EE[4], msg.O_T_EE[8]],
        #                           [msg.O_T_EE[1], msg.O_T_EE[5], msg.O_T_EE[9]],
        #                           [msg.O_T_EE[2], msg.O_T_EE[6], msg.O_T_EE[10]]])
        arr_frame1_b = np.array([[0.0], [msg.O_T_EE[12]], [msg.O_T_EE[14]]])
        arr_frame1_A = np.matrix([[msg.O_T_EE[0], 0, msg.O_T_EE[8]],
                                              [0, 1, 0],
                                  [msg.O_T_EE[2], 0, msg.O_T_EE[10]]])    

    # Preparing the b matrix & A matrix for frame 2 (Position & RotationMatrix of the Ending point(frame) of the Demon w.r.t the base frame of the robot (panda_link0))
    # arr_frame2_b = np.array([[msg.O_T_EE[12]], [msg.O_T_EE[13]], [msg.O_T_EE[14]]])
    # arr_frame2_A = np.matrix([[msg.O_T_EE[0], msg.O_T_EE[4], msg.O_T_EE[8]],
    #                           [msg.O_T_EE[1], msg.O_T_EE[5], msg.O_T_EE[9]],
    #                           [msg.O_T_EE[2], msg.O_T_EE[6], msg.O_T_EE[10]]])
    arr_frame2_b = np.array([[0.0], [msg.O_T_EE[12]], [msg.O_T_EE[14]]])
    arr_frame2_A = np.matrix([[msg.O_T_EE[0], 0, msg.O_T_EE[8]],
                                          [0, 1, 0],
                              [msg.O_T_EE[2], 0, msg.O_T_EE[10]]])
            
    msg_count += 1
demon_bag.close()
print("Number of data points in Demo {}: {}".format(task_num, msg_count))

# Filling in b matrix & A matrix for frame 1
conc_arr_frame1_b = np.ones((3, msg_count)) * arr_frame1_b
conc_arr_frame1_A = np.tile(arr_frame1_A.T, msg_count) # the .T is because the rotation is around negative y-axis

# Filling in b matrix & A matrix for frame 2
conc_arr_frame2_b = np.ones((3, msg_count)) * arr_frame2_b
conc_arr_frame2_A = np.tile(arr_frame2_A.T, msg_count)

# Saving in txt files
np.savetxt(TPGMM_dir + "Demon" + task_num + "_sample_Data.txt", conc_arr_Data[:,1:], fmt='%.5f') # cut out the 1st column (zeros) that was created only to initialize the array 
np.savetxt(TPGMM_dir + "Demon" + task_num + "_sample_frame1_b.txt", conc_arr_frame1_b, fmt='%.5f')
np.savetxt(TPGMM_dir + "Demon" + task_num + "_sample_frame1_A.txt", conc_arr_frame1_A, fmt='%.5f')
np.savetxt(TPGMM_dir + "Demon" + task_num + "_sample_frame2_b.txt", conc_arr_frame2_b, fmt='%.5f')
np.savetxt(TPGMM_dir + "Demon" + task_num + "_sample_frame2_A.txt", conc_arr_frame2_A, fmt='%.5f')

# Add the ',' to put file in the format that TP-GMM code accepts
with open(r"{}".format(TPGMM_dir) + "Demon" + task_num + "_sample_Data.txt", 'r') as f: # The 'r' before the directory name is to open the file as read-only
    data = f.read(); data = data.replace(' ', ',')
with open(r"{}".format(TPGMM_dir) + "Demon" + task_num + "_sample_Data.txt", 'w') as f:
    f.write(data)
    
with open(r"{}".format(TPGMM_dir) + "Demon" + task_num + "_sample_frame1_b.txt", 'r') as f:
    data = f.read(); data = data.replace(' ', ',')
with open(r"{}".format(TPGMM_dir) + "Demon" + task_num + "_sample_frame1_b.txt", 'w') as f:
    f.write(data)

with open(r"{}".format(TPGMM_dir) + "Demon" + task_num + "_sample_frame1_A.txt", 'r') as f:
    data = f.read(); data = data.replace(' ', ',')
with open(r"{}".format(TPGMM_dir) + "Demon" + task_num + "_sample_frame1_A.txt", 'w') as f:
    f.write(data)

with open(r"{}".format(TPGMM_dir) + "Demon" + task_num + "_sample_frame2_b.txt", 'r') as f:
    data = f.read(); data = data.replace(' ', ',')
with open(r"{}".format(TPGMM_dir) + "Demon" + task_num + "_sample_frame2_b.txt", 'w') as f:
    f.write(data)

with open(r"{}".format(TPGMM_dir) + "Demon" + task_num + "_sample_frame2_A.txt", 'r') as f:
    data = f.read(); data = data.replace(' ', ',')
with open(r"{}".format(TPGMM_dir) + "Demon" + task_num + "_sample_frame2_A.txt", 'w') as f:
    f.write(data)

Number of data points in Demo 3: 851


### 3D

In [14]:
task_name = "Rcover"
task_num = str(1)

demon_bag = rosbag.Bag(main_dir + "Demons/" + task_name + "/action/Franka_Stage2_" + task_name + "_action_Demon" + task_num + ".bag", 'r')

data_topic = "/panda_teleop/follower_state_controller/franka_states"

conc_arr_Data = np.zeros((4,1))
msg_count = 0
for topic, msg, t in demon_bag.read_messages(data_topic):

    # Preparing the Data points matrix from the Demons
    # arr_Data = np.array([[msg.O_T_EE[12]], [msg.O_T_EE[13]], [msg.O_T_EE[14]]]) # TODO: add the erro_pose value
    # arr_Data = np.array([[sqrt(msg.O_T_EE_c[12]**2 + msg.O_T_EE_c[13]**2 + msg.O_T_EE_c[14]**2)], [msg.O_T_EE[12]], [msg.O_T_EE[13]], [msg.O_T_EE[14]]]) # Euc. distance of error_pose
    arr_Data = np.array([[msg.O_T_EE_c[0]], [msg.O_T_EE[12]], [msg.O_T_EE[13]], [msg.O_T_EE[14]]]) # Euc. distance of error_pose

    conc_arr_Data = np.concatenate((conc_arr_Data, arr_Data), axis=1)

    # Preparing the b matrix & A matrix for frame 1 (Position & RotationMatrix of the Starting point(frame) of the Demon w.r.t the base frame of the robot (panda_link0))
    if (msg_count == 0):
        arr_frame1_b = np.array([[0], [msg.O_T_EE[12]], [msg.O_T_EE[13]], [msg.O_T_EE[14]]])
        arr_frame1_A = np.matrix([[1,             0,             0,             0],
                                  [0, msg.O_T_EE[0], msg.O_T_EE[4], msg.O_T_EE[8]],
                                  [0, msg.O_T_EE[1], msg.O_T_EE[5], msg.O_T_EE[9]],
                                  [0, msg.O_T_EE[2], msg.O_T_EE[6], msg.O_T_EE[10]]])

    # Preparing the b matrix & A matrix for frame 2 (Position & RotationMatrix of the Ending point(frame) of the Demon w.r.t the base frame of the robot (panda_link0))
    arr_frame2_b = np.array([[0], [msg.O_T_EE[12]], [msg.O_T_EE[13]], [msg.O_T_EE[14]]])
    arr_frame2_A = np.matrix([[1,             0,             0,             0],
                              [0, msg.O_T_EE[0], msg.O_T_EE[4], msg.O_T_EE[8]],
                              [0, msg.O_T_EE[1], msg.O_T_EE[5], msg.O_T_EE[9]],
                              [0, msg.O_T_EE[2], msg.O_T_EE[6], msg.O_T_EE[10]]])
            
    msg_count += 1
demon_bag.close()
print("Number of data points in Demo {}: {}".format(task_num, msg_count))

# Filling in b matrix & A matrix for frame 1
conc_arr_frame1_b = np.ones((4, msg_count)) * arr_frame1_b
conc_arr_frame1_A = np.tile(arr_frame1_A, msg_count) # the .T is because the rotation is around negative y-axis

# Filling in b matrix & A matrix for frame 2
conc_arr_frame2_b = np.ones((4, msg_count)) * arr_frame2_b
conc_arr_frame2_A = np.tile(arr_frame2_A, msg_count)

# Saving in txt files
np.savetxt(TPGMM_dir + "Demon" + task_num + "_sample_Data.txt", conc_arr_Data[:,1:], fmt='%.5f') # cut out the 1st column (zeros) that was created only to initialize the array 
np.savetxt(TPGMM_dir + "Demon" + task_num + "_sample_frame1_b.txt", conc_arr_frame1_b, fmt='%.5f')
np.savetxt(TPGMM_dir + "Demon" + task_num + "_sample_frame1_A.txt", conc_arr_frame1_A, fmt='%.5f')
np.savetxt(TPGMM_dir + "Demon" + task_num + "_sample_frame2_b.txt", conc_arr_frame2_b, fmt='%.5f')
np.savetxt(TPGMM_dir + "Demon" + task_num + "_sample_frame2_A.txt", conc_arr_frame2_A, fmt='%.5f')

# Add the ',' to put file in the format that TP-GMM code accepts
with open(r"{}".format(TPGMM_dir) + "Demon" + task_num + "_sample_Data.txt", 'r') as f: # The 'r' before the directory name is to open the file as read-only
    data = f.read(); data = data.replace(' ', ',')
with open(r"{}".format(TPGMM_dir) + "Demon" + task_num + "_sample_Data.txt", 'w') as f:
    f.write(data)
    
with open(r"{}".format(TPGMM_dir) + "Demon" + task_num + "_sample_frame1_b.txt", 'r') as f:
    data = f.read(); data = data.replace(' ', ',')
with open(r"{}".format(TPGMM_dir) + "Demon" + task_num + "_sample_frame1_b.txt", 'w') as f:
    f.write(data)

with open(r"{}".format(TPGMM_dir) + "Demon" + task_num + "_sample_frame1_A.txt", 'r') as f:
    data = f.read(); data = data.replace(' ', ',')
with open(r"{}".format(TPGMM_dir) + "Demon" + task_num + "_sample_frame1_A.txt", 'w') as f:
    f.write(data)

with open(r"{}".format(TPGMM_dir) + "Demon" + task_num + "_sample_frame2_b.txt", 'r') as f:
    data = f.read(); data = data.replace(' ', ',')
with open(r"{}".format(TPGMM_dir) + "Demon" + task_num + "_sample_frame2_b.txt", 'w') as f:
    f.write(data)

with open(r"{}".format(TPGMM_dir) + "Demon" + task_num + "_sample_frame2_A.txt", 'r') as f:
    data = f.read(); data = data.replace(' ', ',')
with open(r"{}".format(TPGMM_dir) + "Demon" + task_num + "_sample_frame2_A.txt", 'w') as f:
    f.write(data)

Number of data points in Demo 1: 1455


###

In [73]:
temp_arr_Data = np.loadtxt(TPGMM_dir + "Demon" + task_num + "_sample_Data.txt", delimiter=',')
print(temp_arr_Data.shape)

temp_arr_Data = np.loadtxt(TPGMM_dir + "Demon" + task_num + "_sample_frame1_A.txt", delimiter=',')
print(temp_arr_Data)

with open(TPGMM_dir + "Demon" + task_num + "_sample_frame1_A.txt", 'r') as f:
    print(f.read())

(3, 851)
[[ 0.99986  0.       0.00331 ...  0.99986  0.       0.00331]
 [ 0.       1.       0.      ...  0.       1.       0.     ]
 [ 0.00337  0.      -0.99999 ...  0.00337  0.      -0.99999]]
0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000,0.00331,0.99986,0.00000